In [1]:
import random

In [2]:
# define region
REGION_NOT_USE = 0
REGION_TYPE_SUPER = 1
REGION_TYPE_DIR = 2
REGION_TYPE_FILE = 3
REGION_TYPE_RESERVE = 4
region_type = [REGION_NOT_USE] * 128
region_type[0] = REGION_TYPE_SUPER
region_type[1] = REGION_TYPE_DIR
region_type[2] = REGION_TYPE_FILE
region_type[127] = REGION_TYPE_RESERVE

# TODO
# a sector can only store 2 maps
# id map主要作用是将superblock与后面的东东对齐，大小是变化的
MAP_CNT_THRESHOLD = 4
WL_CNT_THRESHOLD = 4
ID_CNT_THRESHOLD = 2
CNT_BEGIN_THRESHOLD = 2 + ID_CNT_THRESHOLD

# define sector map
# init, superblock 2
SECTOR_FREE = 1
SECTOR_USING = 0
sector_map = [SECTOR_FREE] * 8192
# TODO
# sector map sector也要using
for i in range(2 + ID_CNT_THRESHOLD + MAP_CNT_THRESHOLD):
    sector_map[i] = SECTOR_USING

# TODO
sector_map_sector = 2 + ID_CNT_THRESHOLD
sector_map_cnt = 1
# TODO
id_map_sector = 2
id_map_cnt = 1

# TODO
# super region record
super_region_off = CNT_BEGIN_THRESHOLD + MAP_CNT_THRESHOLD

# TODO
wl_threshold = 1000

wl_sector = -1
wl_cnt = -1
ldb_sector = -1
ldb_off = -1

# erase times of all sectors
etimes = [0] * 8192
scan_times = 0

# The sort module of wear leveling
# (region, etimes, type)
sort_wl = []

# dir region record
dir_region = 1
dir_off = 0

# bfile region record
bfile_region = 2
bfile_off = 0

# reserve region record
reserve_region = 127
reserve_off = 0

# record the number of sector_map_sector, ldb_num, wl_sector_num, root_dir_num
# id_num can cal by myself
sector_map_sector_num = 0
ldb_sector_num = 0
wl_sector_num = 0
root_dir_sector_num = 0

# the sector of root dir
root_sector = -1

In [3]:
def new_sector_in_super_region(num):
    global super_region_off
    global sector_map

    loop = num
    out = -1
    alert = 0
    while(loop > 0):
        if (super_region_off >= 64):
            # TODO
            super_region_off = CNT_BEGIN_THRESHOLD
        if out != -1 and sector_map[super_region_off]!=SECTOR_FREE:
            # print(out, super_region_off)
            assert(-1 > 0)
        if(sector_map[super_region_off] == SECTOR_FREE):
            # alloc sector for wl module
            if loop == num:
                out = super_region_off
                # print(out)
                assert (out - CNT_BEGIN_THRESHOLD) % MAP_CNT_THRESHOLD == 0
            sector_map[super_region_off] = SECTOR_USING
            loop = loop - 1
        super_region_off += 1
        alert += 1
        assert(alert < 100)
    
    return out

def free_sector(sector):
    global sector_map
    global etimes

    # print(sector)
    assert(sector_map[sector] == SECTOR_USING)
    sector_map[sector] = SECTOR_FREE
    etimes[sector] += 1

In [4]:
def using_reserve_sector():
    global etimes
    global reserve_region
    global reserve_off

    etimes[reserve_region * 64 + reserve_off] += 1
    reserve_off += 1
    if reserve_off == 64:
        reserve_off = 0

In [5]:
def exchange_region(begin, end):
    global sort_wl
    global region_type
    global sector_map
    global etimes
    global root_sector
    
    # exchange region type
    first = sort_wl[begin][0]
    second = sort_wl[end][0]
    temp = region_type[first]
    region_type[first] = region_type[second]
    region_type[second] = temp

    # exchange region type in sort_wl
    temp = sort_wl[begin][2]
    sort_wl[begin][2] = sort_wl[end][2]
    sort_wl[end][2] = temp
    
    # exchange each sector
    for i in range(64):
        index_a = first * 64 + i
        index_b = second * 64 + i

        # the root sector should also be changed
        if root_sector == index_a:
            root_sector = index_b
        elif root_sector == index_b:
            root_sector = index_a
        
        if((sector_map[index_a] == SECTOR_FREE) and (sector_map[index_b] == SECTOR_FREE)):
            pass
        elif((sector_map[index_a] == SECTOR_FREE) and (sector_map[index_b] == SECTOR_USING)):
            # update data after exchanging
            etimes[index_b] += 1
            sector_map[index_a] = SECTOR_USING
            sector_map[index_b] = SECTOR_FREE
        elif((sector_map[index_a] == SECTOR_USING) and (sector_map[index_b] == SECTOR_FREE)):
            etimes[index_a] += 1
            sector_map[index_a] = SECTOR_FREE
            sector_map[index_b] = SECTOR_USING
        elif((sector_map[index_a] == SECTOR_USING) and (sector_map[index_b] == SECTOR_USING)):
            etimes[index_a] += 1
            etimes[index_b] += 1
            # use a reserve sector
            using_reserve_sector()

In [6]:
def construct_wl():
    global sort_wl
    global etimes
    global region_type
    global dir_region
    global dir_off
    global bfile_region
    global bfile_off

    # count and sort erase times of each region
    sort_wl = []
    for i in range(128):
        count = 0
        for j in range(64):
            count = count + etimes[i * 64 + j]
        sort_wl.append([i, count, region_type[i]])
    sort_wl.sort(key= lambda x: x[1])

    # migrate data among regions
    begin = 0
    end = 127
    while begin < end:
        if (sort_wl[begin][2] == REGION_TYPE_SUPER) or (sort_wl[begin][2] == REGION_TYPE_RESERVE):
            begin += 1
            continue
        elif (sort_wl[end][2] == REGION_TYPE_SUPER) or (sort_wl[end][2] == REGION_TYPE_RESERVE):
            end -= 1
            continue
        
        exchange_region(begin, end)
        begin += 1
        end -= 1

    # get new dir region
    for i in range(128):
        if (sort_wl[i][2] == REGION_TYPE_DIR):
            dir_region = sort_wl[i][0]
            dir_off = 0
            break
    
    # get new file region
    for i in range(128):
        if (sort_wl[i][2] == REGION_TYPE_FILE):
            bfile_region = sort_wl[i][0]
            bfile_off = 0
            break
    

In [7]:
def next_region_nowl(my_region_type, current_region):
    global reserve_region
    global scan_times
    global sector_map_cnt
    global sector_map
    global sector_map_sector
    global etimes
    global wl_threshold
    global wl_sector
    global wl_cnt
    global ldb_sector
    global ldb_off
    global dir_region
    global bfile_region
    global region_type
    global sector_map_sector_num
    global wl_sector_num
    global ldb_sector_num

    while(True):
        # get the next region
        current_region = (current_region + 1) % 128

        # skip reserve region
        if (current_region == reserve_region):
            continue

        # has scanned nor flash once
        if (current_region == 0):
            # update data
            scan_times += 1
            sector_map_cnt += 1

            # change a sector to store sector map
            if (sector_map_cnt == MAP_CNT_THRESHOLD):
                # turn old sector to free
                # print(sector_map_sector)
                assert(sector_map[sector_map_sector] == SECTOR_USING)
                assert (sector_map_sector - CNT_BEGIN_THRESHOLD) % MAP_CNT_THRESHOLD == 0
                for i in range(sector_map_sector, sector_map_sector+MAP_CNT_THRESHOLD):
                    # print("free " + str(sector_map_sector) + "  " + str(i))
                    # TODO
                    if i >= 64:
                        i = i - CNT_BEGIN_THRESHOLD
                    free_sector(i)

                # find a new sector
                sector_map_sector_num += 1
                sector_map_sector = new_sector_in_super_region(MAP_CNT_THRESHOLD)
                sector_map_cnt = 1
            
            if (scan_times == wl_threshold):
                # new sector for wl array
                wl_sector_num += 1
                wl_sector = new_sector_in_super_region(WL_CNT_THRESHOLD)
                wl_cnt = 1

                # new sector for ldb message
                ldb_sector_num += 1
                ldb_sector = new_sector_in_super_region(MAP_CNT_THRESHOLD)
                ldb_off = 0

                scan_times += 1
                construct_wl()
                if (my_region_type == REGION_TYPE_DIR):
                    return dir_region
                elif (my_region_type == REGION_TYPE_FILE):
                    return bfile_region
                else:
                    assert(-1 > 0)
            
            # super region is not we want, continue
            assert(my_region_type != REGION_TYPE_SUPER)
            continue

        # Find the next region of type
        if region_type[current_region] == REGION_NOT_USE:
            region_type[current_region] = my_region_type
            return current_region
        elif region_type[current_region] == my_region_type:
            return current_region

In [8]:
def next_region_wl(my_region_type, current_region):
    global ldb_off
    global ldb_sector
    global sector_map_cnt
    global sector_map_sector
    global wl_cnt
    global wl_sector
    global dir_region
    global bfile_region
    global sort_wl
    global ldb_sector_num
    global sector_map_sector_num
    global wl_sector_num
    
    # update ldb message
    ldb_off += 8
    if ldb_off >= 4096 * MAP_CNT_THRESHOLD:
        # get a new sector to store ldb
        ldb_sector_num += 1
        assert (ldb_sector - CNT_BEGIN_THRESHOLD) % MAP_CNT_THRESHOLD == 0
        for i in range(ldb_sector, ldb_sector+MAP_CNT_THRESHOLD):
            # print("free " + i)
            # TODO
            if i >= 64:
                i = i - CNT_BEGIN_THRESHOLD
            free_sector(i)
        ldb_sector = new_sector_in_super_region(MAP_CNT_THRESHOLD)
        ldb_off = 0

        # get a new sector to store sector map
        sector_map_cnt += 1
        if sector_map_cnt >= MAP_CNT_THRESHOLD:
            sector_map_sector_num += 1
            assert (sector_map_sector - CNT_BEGIN_THRESHOLD) % MAP_CNT_THRESHOLD == 0
            for i in range(sector_map_sector, sector_map_sector+MAP_CNT_THRESHOLD):
                # print("free " + i)
                # TODO
                if i >= 64:
                    i = i - CNT_BEGIN_THRESHOLD
                free_sector(i)
            sector_map_sector = new_sector_in_super_region(MAP_CNT_THRESHOLD)
            sector_map_cnt = 1
        
        # get a new sector to store wl array
        wl_cnt += 1
        if wl_cnt > WL_CNT_THRESHOLD:
            wl_sector_num += 1
            assert (wl_sector - CNT_BEGIN_THRESHOLD) % MAP_CNT_THRESHOLD == 0
            for i in range(wl_sector, wl_sector+WL_CNT_THRESHOLD):
                # print("free " + i)
                # TODO
                if i >= 64:
                    i = i - CNT_BEGIN_THRESHOLD
                free_sector(i)
            wl_sector = new_sector_in_super_region(WL_CNT_THRESHOLD)
            wl_cnt = 1
            construct_wl()

            if (my_region_type == REGION_TYPE_DIR):
                return dir_region
            elif (my_region_type == REGION_TYPE_FILE):
                return bfile_region
            else:
                assert(-1 > 0)

    # find the next region to use
    cnt = 0
    first_region = -1
    end_flag = False
    # TODO
    for i in range(256):
        if (sort_wl[i%128][2] == my_region_type):
            if cnt == 0:
                # record the first region, if current region is to WL_CNT_THRESHOLD, return to use this
                first_region = sort_wl[i%128][0]

                # 不能找到奇奇怪怪的东西
                if sector_map[first_region * 64] == SECTOR_USING:
                    first_region = -1
            if end_flag == True:
                # the region after current region
                return sort_wl[i%128][0]
            cnt += 1
        if (sort_wl[i%128][0] == current_region):
            # record current region
            end_flag = True

            # TODO
            # print(i, end_flag, first_region, current_region)

            # TODO,问题在这里，region数量不能太少，同时region不能是存放binary的东东，不然没有空间
            if cnt == 4 and first_region != -1:
                return first_region
    
    assert(-1 > 0)

In [9]:
def next_region(my_region_type, current_region):
    global scan_times
    global wl_threshold

    if(scan_times <= wl_threshold):
        return next_region_nowl(my_region_type, current_region)
    else:
        return next_region_wl(my_region_type, current_region)

In [10]:
# No wl version
def my_wl_cnt(ssize, snum, bsize, loop):
    global bfile_off
    global bfile_region
    global sector_map
    global dir_region
    global dir_off
    global sector_map
    global etimes
    global id_map_sector
    global root_dir_sector_num
    global sector_map_sector_num
    global wl_sector_num
    global ldb_sector_num

    # 12 is head message, 8 is skip message, rest is what we can use
    valid_space_in_sector = 4096 - 12

    # Write big file first
    big_size = bsize
    while big_size > 0:
        temp_size = valid_space_in_sector if (big_size > valid_space_in_sector) else big_size
        if(bfile_off == 64):
            bfile_region = next_region(REGION_TYPE_FILE, bfile_region)
            bfile_off = 0
        if(sector_map[bfile_region * 64 + bfile_off] == SECTOR_FREE):
            sector_map[bfile_region * 64 + bfile_off] = SECTOR_USING
            big_size -= temp_size
        bfile_off += 1

    #  Write/Delete small file
    for i in range(loop):

        if (i % 10000 == 9999):
            print("loop " + str(i+1))

        sfile_num = snum
        # the name and big file index with header
        bfile_name_index_size= 4 + 12 + 4 + 12
        sfile_name_size = 4 + 12
        sfile_data_size = 4 + ssize
        next_flag = False
        while sfile_num > 0:
            temp_num = 0

            # GC for every 4 sectors, should migrate the bfile name and index
            if dir_off % 4 == 0:
                root_dir_sector_num += 1
                temp_size = valid_space_in_sector - bfile_name_index_size
            else:
                temp_size = valid_space_in_sector
            
            # left a small file data in the sector to be written
            if (next_flag == True):
                next_flag = False
                temp_num += 1
                temp_size -= sfile_data_size

            # the number of small files that can be proged in the sector
            temp_num += temp_size // (sfile_name_size + sfile_data_size)

            # can prog a name without data
            if temp_size % (sfile_name_size + sfile_data_size) > sfile_name_size:
                next_flag = True

            # write small files
            current_sector = dir_region * 64 + dir_off
            if (sector_map[current_sector] == SECTOR_FREE):
                sector_map[current_sector] = SECTOR_FREE
                etimes[current_sector] += 1
                sfile_num -= temp_num
            else:
                # current sector is not valid, something has been wrong
                assert(-1 > 0)

            # update sector map
            dir_off += 1
            if (dir_off == 64):
                dir_region = next_region(REGION_TYPE_DIR, dir_region)
                dir_off = 0
    
    # update the id map message
    id_map_num = loop // 8192 + 1
    id_map_sector_num = id_map_num // 2 + 1
    id_map_sector_off = 2
    print("id_map_sector_num " + str(id_map_sector_num))
    for i in range(int(id_map_sector_num)):
        # id_map_sector = new_sector_in_super_region(1)
        # free_sector(id_map_sector)
        # print("free " + i)
        etimes[id_map_sector_off] += 1
        id_map_sector_off = id_map_sector_off + 1
        if id_map_sector_off >= CNT_BEGIN_THRESHOLD:
            id_map_sector_off = 2

    # update the superblock message
    total_size = 16 * sector_map_sector_num + 16 * id_map_sector_num + 16 * wl_sector_num + 8 * ldb_sector_num + 8 * root_dir_sector_num
    rest_size_in_superblock = 4096 - 4 - 36 - 36
    super_erase_times = total_size // rest_size_in_superblock + 1
    etimes[1] = int(super_erase_times // 2)
    etimes[0] = int(super_erase_times // 2 if (super_erase_times % 2 == 0) else super_erase_times // 2 + 1)

In [11]:
# my_wl_cnt(128 * 1024, 128 * 1024, 100000)
my_wl_cnt(32, 16 * 1024, 16 * 1024 * 1024, 1000000)
f=open("./N2FS_sfile_1000000.txt","w")
f.write(str(etimes))
f.close()

loop 10000
loop 20000
loop 30000
loop 40000
loop 50000
loop 60000
loop 70000
loop 80000
loop 90000
loop 100000
loop 110000
loop 120000
loop 130000
loop 140000
loop 150000
loop 160000
loop 170000
loop 180000
loop 190000
loop 200000
loop 210000
loop 220000
loop 230000
loop 240000
loop 250000
loop 260000
loop 270000
loop 280000
loop 290000
loop 300000
loop 310000
loop 320000
loop 330000
loop 340000
loop 350000
loop 360000
loop 370000
loop 380000
loop 390000
loop 400000
loop 410000
loop 420000
loop 430000
loop 440000
loop 450000
loop 460000
loop 470000
loop 480000
loop 490000
loop 500000
loop 510000
loop 520000
loop 530000
loop 540000
loop 550000
loop 560000
loop 570000
loop 580000
loop 590000
loop 600000
loop 610000
loop 620000
loop 630000
loop 640000
loop 650000
loop 660000
loop 670000
loop 680000
loop 690000
loop 700000
loop 710000
loop 720000
loop 730000
loop 740000
loop 750000
loop 760000
loop 770000
loop 780000
loop 790000
loop 800000
loop 810000
loop 820000
loop 830000
loop 840000
l

In [12]:
print(sum(etimes)/len(etimes))
print(max(etimes))
print(sector_map_sector_num + wl_sector_num + ldb_sector_num + root_dir_sector_num)

26012.432373046875
52492
52752837
